# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'udacity-azure-capstone-project'

experiment=Experiment(ws, experiment_name)

In [3]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()
print(ws)

Workspace name: quick-starts-ws-143110
Azure region: southcentralus
Subscription id: 6b4af8be-9931-443e-90f6-c4c34a1f9737
Resource group: aml-quickstarts-143110
Workspace.create(name='quick-starts-ws-143110', subscription_id='6b4af8be-9931-443e-90f6-c4c34a1f9737', resource_group='aml-quickstarts-143110')


In [4]:
compute_cluster_name = "Saturn-AI-Lab"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
except ComputeTargetException:
    compute_cluster_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_cluster_config)

compute_cluster.wait_for_completion(show_output=True)

Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
DATASET_URI = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'

In [6]:
key = 'Heart Failure Clinical Records'

if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("Dataset exists in the datastore.")

else:
    dataset = Dataset.Tabular.from_delimited_files(DATASET_URI)
    dataset = dataset.register(ws,key)

df = dataset.to_pandas_dataframe()
print(f'Dataframe shape:', df.shape)

train, test = train_test_split(df, test_size=.33, shuffle=True)
print(f'Train count: ', len(train))
print(f'Test count: ', len(test))
train.to_csv('train.csv',index = False)
test.to_csv('test.csv',index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])
datastore.upload_files(files = ['./test.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore,'train.csv')])
test = Dataset.Tabular.from_delimited_files([(datastore,'test.csv')])

Dataset exists in the datastore.
Dataframe shape: (299, 13)
Train count:  200
Test count:  99
Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files
Uploading an estimated of 1 files
Target already exists. Skipping upload for test.csv
Uploaded 0 files


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, delay_evaluation=5, slack_factor = 0.2)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'C': uniform(0, 20),
                                        'max_iter' : quniform(10, 250, 1),
                                        'solver' : choice('liblinear','sag','lbfgs', 'saga')})

if "training" not in os.listdir():
    os.mkdir("./training")


#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(entry_script="train.py", compute_target=compute_cluster, source_directory="./")

hyperdrive_run_config = HyperDriveConfig(
    estimator=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100,
    max_concurrent_runs=4
)

In [ ]:
#TODO: Submit your experiment

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service